In [3]:
from tensorflow.keras import layers, models, Model, Input
import numpy as np
import tensorflow as tf
import os

# Load all models from the folder
models_folder = "Models_withLHAPDF"
model_files = [f for f in os.listdir(models_folder) if f.endswith(".h5")]

models_list = [tf.keras.models.load_model(os.path.join(models_folder, f)) for f in model_files]

# Generate predictions for a range of QM values
QM_values = np.linspace(1, 10, 500).reshape(-1, 1)  # Ensure input shape matches the model
dnn_outputs = np.array([model.predict(QM_values, verbose=0).flatten() for model in models_list])

# Compute mean and standard deviation
mean_values = np.mean(dnn_outputs, axis=0)
std_values = np.std(dnn_outputs, axis=0)

# Create a non-trainable model that returns mean and std
def DNNQ_mean_std(QM_values, mean_values, std_values):
    """ Creates a non-trainable model that outputs stored mean and std values """
    input_layer = Input(shape=(1,))
    mean_layer = layers.Embedding(input_dim=len(QM_values), output_dim=1, input_length=1, trainable=False)
    std_layer = layers.Embedding(input_dim=len(QM_values), output_dim=1, input_length=1, trainable=False)
    
    # Assign precomputed mean and std as embeddings
    mean_layer.build((None,))
    std_layer.build((None,))
    
    mean_layer.set_weights([mean_values.reshape(-1, 1)])
    std_layer.set_weights([std_values.reshape(-1, 1)])

    mean_output = mean_layer(input_layer)
    std_output = std_layer(input_layer)

    return Model(inputs=input_layer, outputs=[mean_output, std_output])

# Create the model
single_model = DNNQ_mean_std(QM_values, mean_values, std_values)
print("Single non-trainable model is ready!")

# Save the single model
single_model.save("single_model_with_mean_std.h5")
print("Single model saved successfully!")


Single non-trainable model is ready!
Single model saved successfully!
